# Variables
#### This cell will create in the current directory two symbolic links named "PS" and "WS".
- PS - project space. Here you can store your raw data and results. It is long storage.
- WS - working space. Here you should run your analyses. It is short storage.
- RR - folder for your raw reads. Copy your raw reads to this folder!
- CR - folder for cleaned reads. After cleaning your samples will be stored there.
- DB - folder for databases. Symbolic link will be created in your "Home" directory.
- REFS - folder for reference genomes.
- LOGS - folder for all Binac2 logs to keep.
- BIN - folder for additional scripts.
#### ATTENTION: Pandas should be installed in your base environment:
- conda install -c conda-forge -c bioconda pandas
#### !!! Move the results that you want to keep from "WS" to "PS" when you are done!!!


In [1]:
# Name for the project / work spaces and your Binac2 user name
NAME = 'LimBiom'
USER = 'user_name'
K2DB = 'nt' #kraken2 db, check options "qiime annotate build-kraken-db", 
              #and description: https://benlangmead.github.io/aws-indexes/k2
 
# Variables that will be shared between this notebook and all bash scripts
VARS = (
    f'WS=/pfs/10/work/{USER}-{NAME}/\n'
    f'PS=/pfs/10/project/bw18d010/{USER}/{NAME}/\n'
    f'DB=/pfs/10/project/bw18d010/{USER}/Databases/\n'
    f'REFS=/pfs/10/project/bw18d010/{USER}/Databases/Reference_Genomes\n'
    f'LOGS=/pfs/10/project/bw18d010/{USER}/{NAME}/Logs\n'
    f'RR=/pfs/10/project/bw18d010/{USER}/{NAME}/Raw_reads\n'
    f'CR=/pfs/10/project/bw18d010/{USER}/{NAME}/Clean_reads\n'
    f'BIN=~/Scripts\n'
)

# Path to your conda 
CONDA = '$HOME/miniforge3/etc/profile.d/conda.sh'

###############################################################################
################################ DO NOT CHANGE ################################
###############################################################################
import pandas as pd
import numpy as np
import os
from uuid import uuid4

#extract bash variables for python
for var in VARS.strip().split('\n'):
    vars()[var.split('=')[0]]=var.split('=')[1]

#create folders and symbolic links
!mkdir -p $WS $PS $DB $REFS $RR $CR $BIN $LOGS
!ln -sf -T {PS} ./'PS'
!ln -sf -T {WS} ./'WS'
!ln -sf -T {DB} ~/'DB'

#Binac2 function
source = f'#!/bin/bash\nsource {CONDA}'
COMM = 'export TMPDIR=$TMPDIR\n#Custom variables and commands\ncd $WS\n'
STRING = '\n'.join([source, VARS, COMM])

# binac function that will submit the job to server
def binac2(tasks=1, ppn=1, time='00:15:00', mem=2, name='myjob', logdir=LOGS,
          queue='compute', todo='echo "Command"', STRING=STRING, nodes=1):
    text = '\n'.join([STRING, todo])
    script = f'{logdir}/{name}.sh'
    !mkdir -p {logdir}
    with open(script, "w") as text_file:
        text_file.write(text)
    !chmod +x {script}
    run = f"""sbatch -t {time} -p {queue} --mem={mem}g -J {name} --output={logdir}/{name}.log \
              --error={logdir}/{name}.error --ntasks-per-node={tasks} -N {nodes} {script}"""
    !$run

cleaning = !rm -r $WS/*/*/.ipynb_c* $WS/*/.ipynb_c* $WS/.ipynb_c* $PS/*/*/.ipynb_c* $PS/*/.ipynb_c* $PS/.ipynb_c*
FORMAT = "JobId:10,Name:20,MinMemory:11,NumTasks:6,State:9,TimeUsed:12,TimeLimit:11"

###############################################################################
######################## Here you can change something ########################
###############################################################################

# List all raw samples
raws = []
if os.path.exists(RR): 
    raws = sorted(list(set([f.rsplit('_', 1)[0] for f in os.listdir(RR) if f.endswith('.gz')])))
    print('Number of raw samples: ', len(raws))

# List all clean samples
cleans = []
if os.path.exists(CR): 
    cleans = sorted(list(set([f.rsplit('_', 1)[0] for f in os.listdir(CR) if f.endswith('.gz')])))
    print('Number of clean samples: ', len(cleans))

# List all assemblies
spades = []
if os.path.exists(f'{PS}/SPAdes'): 
    spades = sorted(list(set([f.rsplit('_',1)[0] for f in os.listdir(f'{PS}/SPAdes') if f.endswith('.fasta')])))
    print('Number of assemblies: ', len(spades))

Number of raw samples:  0
Number of clean samples:  157
Number of assemblies:  157


# Managing your work space
Here are the command to manage your work space. Uncomment to run

In [ ]:
#!ws_allocate {NAME} 30
#!ws_list -a
#!ws_find {NAME}
#!ws_extend {NAME} 30
#!ws_release {NAME} #delete, remove content first

# Rename samples. 
If your samples contain some extra substrings you need to rename it as SampleID_1/2.fq.gz

In [18]:
# Count "_" in your SampleIDs and set "n" to that number.
# Examples: "My_sample1" - n = 1; "MySample1" - n = 0

n = 1 
for raw in raws:
    new = '_'.join(raw.split('_')[:n+1])

    !mv {RR}/{raw}_1.fq.gz {RR}/{new}_1.fq.gz
    !mv {RR}/{raw}_2.fq.gz {RR}/{new}_2.fq.gz

raws = sorted(list(set([f.rsplit('_', 1)[0] for f in os.listdir(RR) if f.endswith('.gz')])))

# QC and host DNA removal

## Installation
Better run directly in the terminal (uncomment)

In [ ]:
#conda create -n QC -c bioconda -c conda-forge trim-galore bowtie2 -y

## Create index
- Only once. When reusing the same reference genomes, you don't need to recreate.
- Modify "REF-GENS" list to contain only the genomes you need.
- ATTENTION: In your REFS directory, create folder with the same name as the genomes from "REF_GENS" and upload there genome of interest in the ".fna.gz" format.

In [22]:
THREADS = 8
logdir = f'{LOGS}/Bowtie2'

todo = '''
conda activate QC
cd $REFS/{index}

#skip if index exists
if [ -f $REFS/{index}/*.bt2 ]
then
    echo "Index {index} already exists."
else
    pwd
    echo "Creating index {index}..."
    bowtie2-build ./*.fna.gz {index} --threads {THREADS}
fi
'''

# list of genomes to make the index
REF_GENS = ['Chicken', 'Corn', 'Cow', 'Duck', 'Human', 'Pig', 'Turkey']

for index in REF_GENS:
    #if index == REF_GENS[0]:
    #    continue
    
    binac2(tasks=THREADS, time='10:00:00', mem=16, name=f'{index}_BW2_ind', logdir=logdir, 
           todo=todo.format(index=index, THREADS=THREADS))

Submitted batch job 38620
Submitted batch job 38621
Submitted batch job 38622
Submitted batch job 38623
Submitted batch job 38624
Submitted batch job 38625


## Read QC and Host DNA removal
Use batches to process samples in parallel

In [41]:
#Read QC and host DNA removal

THREADS = 8
logdir = f"{LOGS}/QC"

todo = '''
conda activate QC
sras=({IDS})
GENS=({GENS})

echo "Processing {BATCH}"
for name in ${{sras[@]}}; do
    preQC=$PS/FastQC/preQC/$name
    postQC=$PS/FastQC/postQC/$name
    outdir=$CR/$name
    
    echo
    echo "############################################################"
    echo "Processing $name "

    #skip processed files
    if [ -d $outdir ]; then
        echo "$name already busy. Skipping."
        continue
    fi

    cd $WS
    RR1=$RR/${{name}}_1.f*q.gz
    RR2=$RR/${{name}}_2.f*q.gz
    CR1=$CR/${{name}}_1.fq.gz
    CR2=$CR/${{name}}_2.fq.gz

    #skip processed files
    if [ -f $CR1 ]; then
        echo "$name already cleaned. Skipping."
        continue
    fi
    
    rm -rf $outdir $preQC $postQC $CR1 $CR2
    mkdir -p $preQC $postQC $outdir 

    # Fastq report before QC
    echo "Report before QC..."
    fastqc -q -t {THREADS} -o $preQC -f fastq $RR1 $RR2
    
    echo
    echo "Running Trim-Galore for QC"
    echo   
    cd $outdir
    trim_galore --paired $RR1 $RR2 -j {THREADS}

    # Fastq report after QC
    echo "Report after QC..."
    cd
    cd $WS
    R1=$outdir/*_val_1.f*q.gz
    R2=$outdir/*_val_2.f*q.gz
    fastqc -q -t {THREADS} -o $postQC -f fastq $R1 $R2
    
    echo
    
    for GEN in ${{GENS[@]}}; do
        echo "Removing $GEN DNA from reads"
        cd
        cd $REFS/$GEN
        bowtie2 -p {THREADS} -x $GEN -1 $R1 -2 $R2 \
        --un-conc-gz $outdir/no_${{GEN}} > $outdir/${{GEN}}.sam
        echo "$GEN DNA removed"
        R1=$outdir/no_${{GEN}}.1
        R2=$outdir/no_${{GEN}}.2
    done
    
    echo
    echo "Move and clean!"
    cd
    cd $WS
    mv $R1 $CR1
    mv $R2 $CR2
    rm -rf $outdir

    #remove processed files
    if [ -f $CR1 ]; then
        echo "Removing raw read since $name successfully cleaned."
        rm $RR1 $RR2
    fi
done

echo
echo "############################################################"
echo "###                   All done! Enjoy!                   ###"
echo "### Citations:                                           ###"
echo "### 1. Cutadapt: https://doi.org/10.14806/ej.17.1.200    ###"
echo "### 2. Bowtie: https://doi.org/10.1038/nmeth.1923        ###"
echo "### 3. FastQC (optional):                                ###"
echo "###    https://doi.org/10.1038/nmeth.1923                ###"
echo "### 4. TrimGalore (optional):                            ###"
echo "###    https://github.com/FelixKrueger/TrimGalore        ###"
echo "############################################################"
'''

# Split your samples into batches for parallel execution (b - number of samples per batch).
b = 10
batches = {f'b{i+1}-{i+b}': raws[i:i + b] for i in range(0, len(raws), b)}

#launch samples
for batch in batches:
    #if batch not in [f'b1-{b}']:
    #    continue
        
    IDS = ' '.join(batches[batch])
    GENS = ' '.join(['Pig', 'Corn']) #list genomes to remove host DNA
            
    binac2(tasks=THREADS, time='04-00', mem=32, name=f'{batch}_QC_R', logdir=logdir, 
           todo=todo.format(BATCH=batch, IDS=IDS, GENS=GENS, THREADS=THREADS))


Submitted batch job 124003
Submitted batch job 124004
Submitted batch job 124005
Submitted batch job 124006
Submitted batch job 124007
Submitted batch job 124008
Submitted batch job 124009
Submitted batch job 124010
Submitted batch job 124011
Submitted batch job 124012
Submitted batch job 124013
Submitted batch job 124014
Submitted batch job 124015
Submitted batch job 124016
Submitted batch job 124017
Submitted batch job 124018


# Import cleaned reads to Qiime2

In [107]:
# import

THREADS = 4
logdir = f'{LOGS}/Q2'

todo = '''
conda activate qiime2-moshpit-2025.4

mkdir -p TEMP/{BATCH}
export TMPDIR=$WS/TEMP/{BATCH}

echo "Processing {BATCH}"
sras=({IDS})

mkdir -p {BATCH}
for name in ${{sras[@]}}; do
    cp $CR/${{name}}_1.fq.gz {BATCH}/${{name}}_AGCT_L001_R1_001.fastq.gz
    cp $CR/${{name}}_2.fq.gz {BATCH}/${{name}}_AGCT_L001_R2_001.fastq.gz
done

#skip cache if exists
if [ -d ./cache ]; then
    echo "$cache already exists. Skipping."
else
    qiime tools cache-create --cache ./cache    
fi

rm -r {BATCH}/.ipynb_checkpoints
qiime tools cache-import \
    --type 'SampleData[PairedEndSequencesWithQuality]' \
    --input-path {BATCH} \
    --cache ./cache \
    --input-format 'CasavaOneEightSingleLanePerSampleDirFmt' \
    --key {BATCH}

rm -r {BATCH}
'''

# Split your samples into batches for parallel execution (b - number of samples per batch).
b = 10
batches = {f'b{i+1}-{i+b}': cleans[i:i + b] for i in range(0, len(cleans), b)}

#launch samples
for batch in batches:
    #if batch in [f'b1-{b}']:
    #    continue
        
    IDS = ' '.join(batches[batch])
            
    binac2(tasks=THREADS, time='01-00', mem=32, name=f'{batch}_Q2_imp', logdir=logdir, 
           todo=todo.format(BATCH=batch, IDS=IDS))

Submitted batch job 124102
Submitted batch job 124103
Submitted batch job 124104
Submitted batch job 124105
Submitted batch job 124106
Submitted batch job 124107
Submitted batch job 124108
Submitted batch job 124109
Submitted batch job 124110
Submitted batch job 124111
Submitted batch job 124112
Submitted batch job 124113
Submitted batch job 124114
Submitted batch job 124115
Submitted batch job 124116
Submitted batch job 124117


# Taxonomy annotation (Reads)

## Fetch database for Kraken2 (PlusPF)

In [8]:
# database

THREADS = 2
logdir = f'{LOGS}/Q2'

todo = '''
conda activate qiime2-moshpit-2025.4
mkdir -p $WS/TEMP
export TMPDIR=$WS/TEMP

mkdir -p $DB/{K2DB}
qiime annotate build-kraken-db \
    --p-collection {K2DB} \
    --o-kraken2-db $DB/{K2DB}/kraken2.qza \
    --o-bracken-db $DB/{K2DB}/bracken.qza
'''
 
binac2(tasks=THREADS, time='07-00', mem=16, name=f'Q2_db_{K2DB}2', logdir=logdir, 
       todo=todo.format(K2DB=K2DB))

Submitted batch job 499857


## Assign taxonomy (reads) with Kraken2

In [4]:
# taxonomy

THREADS = 24
logdir = f'{LOGS}/Q2'

todo = '''
conda activate qiime2-moshpit-2025.4
mkdir -p TEMP/{BATCH}
export TMPDIR=$WS/TEMP/{BATCH}

echo "Processing {BATCH}"

mkdir -p $PS/Taxonomy $PS/Tables

qiime annotate classify-kraken2 \
    --i-seqs ./cache:{BATCH} \
    --i-db $DB/{K2DB}/kraken2.qza \
    --p-threads {THREADS} \
    --p-confidence 0.1 \
    --p-report-minimizer-data \
    --o-reports ./cache:{BATCH}_reports_reads \
    --o-outputs ./cache:{BATCH}_hits_reads

qiime annotate estimate-bracken \
    --i-kraken2-reports ./cache:{BATCH}_reports_reads \
    --i-db $DB/{K2DB}/bracken.qza \
    --p-threshold 5 \
    --p-read-len 150 \
    --o-taxonomy $PS/Taxonomy/{BATCH}_taxonomy_reads.qza \
    --o-table $PS/Tables/{BATCH}_table_reads.qza \
    --o-reports ./cache:{BATCH}_bracken_reports

rm -r TEMP/{BATCH}
'''

# Split your samples into batches for parallel execution (b - number of samples per batch).
b = 10
batches = {f'b{i+1}-{i+b}': cleans[i:i + b] for i in range(0, len(cleans), b)}

#launch samples
for batch in batches:
    #if batch in [f'b1-{b}']:
    #    continue
                    
    binac2(tasks=THREADS, time='04-00', mem=120, name=f'{batch}_K2_reads', logdir=logdir, 
           todo=todo.format(BATCH=batch, THREADS=THREADS))

Submitted batch job 499699
Submitted batch job 499700
Submitted batch job 499701
Submitted batch job 499702
Submitted batch job 499703
Submitted batch job 499704
Submitted batch job 499705
Submitted batch job 499706
Submitted batch job 499707
Submitted batch job 499708
Submitted batch job 499709
Submitted batch job 499710
Submitted batch job 499711
Submitted batch job 499712
Submitted batch job 499713
Submitted batch job 499714


In [37]:
# taxonomy

THREADS = 24
logdir = f'{LOGS}/Q2'

todo = '''
conda activate qiime2-moshpit-2025.4
mkdir -p TEMP/{BATCH}
export TMPDIR=$WS/TEMP/{BATCH}

echo "Processing {BATCH}"

mkdir -p $PS/Taxonomy_nt $PS/Tables_nt

qiime annotate classify-kraken2 \
    --i-seqs ./cache:{BATCH} \
    --i-db $DB/{K2DB}/kraken2.qza \
    --p-threads {THREADS} \
    --p-confidence 0 \
    --p-report-minimizer-data \
    --o-reports ./cache:{BATCH}_reports_reads_nt \
    --o-outputs ./cache:{BATCH}_hits_reads_nt

qiime annotate estimate-bracken \
    --i-kraken2-reports ./cache:{BATCH}_reports_reads_nt \
    --i-db $DB/{K2DB}/bracken.qza \
    --p-threshold 5 \
    --p-read-len 150 \
    --o-taxonomy $PS/Taxonomy_nt/{BATCH}_taxonomy_reads.qza \
    --o-table $PS/Tables_nt/{BATCH}_table_reads.qza \
    --o-reports ./cache:{BATCH}_bracken_reports_nt

rm -r TEMP/{BATCH}
'''

# Split your samples into batches for parallel execution (b - number of samples per batch).
b = 10
batches = {f'b{i+1}-{i+b}': cleans[i:i + b] for i in range(0, len(cleans), b)}

#launch samples
for batch in batches:
    #if batch in [f'b1-{b}']:
    #    continue
                    
    binac2(tasks=THREADS, time='10-00', mem=1000, name=f'{batch}_K2_reads', logdir=logdir, 
           todo=todo.format(BATCH=batch, THREADS=THREADS, K2DB=K2DB))

Submitted batch job 503037
Submitted batch job 503038
Submitted batch job 503039
Submitted batch job 503040
Submitted batch job 503041
Submitted batch job 503042
Submitted batch job 503043
Submitted batch job 503044
Submitted batch job 503045
Submitted batch job 503046
Submitted batch job 503047
Submitted batch job 503048
Submitted batch job 503049
Submitted batch job 503050
Submitted batch job 503051
Submitted batch job 503052


## Merge and filter tables

In [21]:
# merge taxonomy and tables

THREADS = 2
logdir = f'{LOGS}/Q2'

todo = '''
conda activate qiime2-moshpit-2025.4

qiime feature-table merge \
    --i-tables $PS/Tables/b*-*_table_reads.qza \
    --o-merged-table $PS/Tables/table_reads_all.qza

qiime feature-table merge-taxa \
    --i-data $PS/Taxonomy/b*-*_taxonomy_reads.qza \
    --o-merged-data $PS/Taxonomy/taxonomy_reads.qza

qiime feature-table summarize \
    --i-table $PS/Tables/table_reads_all.qza \
    --o-visualization $PS/Tables/table_reads_all.qzv

rm $PS/Taxonomy/b*-*_taxonomy_reads.qza $PS/Tables/b*-*_table_reads.qza
'''

#launch samples             
binac2(tasks=THREADS, time='01-00', mem=16, name=f'Merging_tables_reads', logdir=logdir, todo=todo)

Submitted batch job 499740


In [3]:
# merge taxonomy and tables

THREADS = 2
logdir = f'{LOGS}/Q2'

todo = '''
conda activate qiime2-moshpit-2025.4

qiime feature-table merge \
    --i-tables $PS/Tables_nt/b*-*_table_reads.qza \
    --o-merged-table $PS/Tables_nt/table_reads_all.qza

qiime feature-table merge-taxa \
    --i-data $PS/Taxonomy_nt/b*-*_taxonomy_reads.qza \
    --o-merged-data $PS/Taxonomy_nt/taxonomy_reads.qza

qiime feature-table summarize \
    --i-table $PS/Tables_nt/table_reads_all.qza \
    --o-visualization $PS/Tables_nt/table_reads_all.qzv

rm $PS/Taxonomy_nt/b*-*_taxonomy_reads.qza $PS/Tables_nt/b*-*_table_reads.qza
'''

#launch samples             
binac2(tasks=THREADS, time='01-00', mem=16, name=f'Merging_tables_reads_nt', logdir=logdir, todo=todo)

Submitted batch job 517964


# Assemblies

## SPAdes

In [144]:
THREADS = 12
logdir = f'{LOGS}/Spades'

todo = '''
conda activate qiime2-moshpit-2025.4

SP=$WS/SPAdes
SPA=$PS/SPAdes

mkdir -p $SP $SPA

echo "Processing {BATCH}"
sras=({IDS})
for name in ${{sras[@]}}; do
    echo
    echo "############################################################"
    echo "Processing $name "
    
    echo "Check if reads already assembled or busy" 
    if [ -f $SPA/${{name}}_scaffolds.fa ]; then
        echo "$name already assembled. Skipping."
        continue
    elif [ -d $SP/$name ]; then
        echo "$name is busy. Skipping." 
        continue
    fi
    echo "$name is not assembled. Attempting assembly..."
    
    CR1=$CR/${{name}}_1.fq.gz
    CR2=$CR/${{name}}_2.fq.gz

    spades.py --meta -1 $CR1 -2 $CR2 -o $SP/$name -t {THREADS}

    echo "Checking if assembly created"
    ass=$SP/$name/scaffolds.fasta
    if [ -f $ass ]; then
        echo "Moving assembly"
        mv $ass $SPA/${{name}}_scaffolds.fasta
        mv $SP/$name/contigs.fasta $SPA/${{name}}_contigs.fasta
    else
        echo "There is something wrong with assembly"
    fi
    #rm -r $SP/$name
done

echo
echo "############################################################"
echo "###                   All done! Enjoy!                   ###"
echo "### Citations:                                           ###"
echo "### 1. Spades: https://doi.org/10.1002/cpbi.102          ###"
echo "############################################################"   
'''

# Split your samples into batches for parallel execution (b - number of samples per batch).
b = 10
batches = {f'b{i+1}-{i+b}': cleans[i:i + b] for i in range(0, len(cleans), b)}

#launch samples
for batch in batches:
    #if batch not in [f'b1-{b}']:
    #    continue
        
    IDS = ' '.join([i for i in batches[batch] if i not in spades])
    if len(IDS) == 0:
        continue
            
    binac2(tasks=THREADS, time='05-00', mem=200, name=f'{batch}_SPAdes', logdir=logdir, 
           todo=todo.format(BATCH=batch, IDS=IDS, THREADS=THREADS))

Submitted batch job 124985


## Quast on assemblies

In [22]:
THREADS = 4
logdir = f'{LOGS}/Quast'

todo = '''
conda activate qiime2-moshpit-2025.4

QUAST=$PS/Quast
SPA=$PS/SPAdes

mkdir -p $QUAST

echo "Processing {BATCH}"
sras=({IDS})
for name in ${{sras[@]}}; do
    echo
    echo "############################################################"
    echo "Processing $name "
    
    echo "Check if output exists" 
    if [ -d $QUAST/$name ]; then
        echo "$name exists. Skipping." 
        continue
    fi

    quast -t {THREADS} -o $QUAST/$name $SPA/${{name}}_contigs.fasta
    
done 
'''

# Split your samples into batches for parallel execution (b - number of samples per batch).
b = 50
batches = {f'b{i+1}-{i+b}': cleans[i:i + b] for i in range(0, len(cleans), b)}

#launch samples
for batch in batches:
    if batch in [f'b1-{b}']:
        continue
        
    IDS = ' '.join(batches[batch])
            
    binac2(tasks=THREADS, time='01-00', mem=16, name=f'{batch}_Quast', logdir=logdir, 
           todo=todo.format(BATCH=batch, IDS=IDS, THREADS=THREADS))

Submitted batch job 125530
Submitted batch job 125531
Submitted batch job 125532
Submitted batch job 125533


In [15]:
#!rm -r {WS}/SPAdes

# MAGs

## COMEbin
#### Installation:
1. In terminal:

- conda create -n comebin_env -c conda-forge -c bioconda comebin

2. Download "print_comment.py" and "gen_cov_file.sh" from their git, and put it to the folder "Scripts" (create if missing) in your $HOME directory.
3. Open "Scripts/gen_cov_file.sh" and modify lines 59-60 to:
- F_reads_suffix=_1.fq.gz
- R_reads_suffix=_2.fq.gz


In [192]:
!chmod +rwx $BIN/*
THREADS = 24
RAM = 120
logdir = f'{LOGS}/COMEbin'

todo = '''
conda activate comebin_env

mkdir -p BAMS $PS/COMEbin

echo "Processing {BATCH}"
sras=({IDS})
for name in ${{sras[@]}}; do
    echo
    echo "############################################################"
    echo "Processing $name "
    
    echo "Check if bins already binned or busy" 
    if [ -d BAMS/$name ]; then
        echo "$name is busy. Skipping." 
        continue
    elif [ -d $PS/COMEbin/$name ]; then
        echo "$name is busy or binned. Skipping." 
        continue
    fi
    echo "$name is not binned. Attempting binning..."
    
    CR1=$CR/${{name}}_1.fq.gz
    CR2=$CR/${{name}}_2.fq.gz
    ASS=$PS/SPAdes/${{name}}_scaffolds.fasta
    BINS=$PS/COMEbin/$name

    echo "Preprocess"
    $BIN/gen_cov_file.sh -a $ASS -o BAMS/$name $CR1 $CR2 -m {RAM} -t {THREADS}
    
    echo "Run COMEBin for bins"
    run_comebin.sh -a $ASS -o $BINS -p BAMS/$name/work_files -t {THREADS}
done

echo
echo "###############################################################"
echo "###                   All done! Enjoy!                      ###"
echo "### Citations:                                              ###"
echo "### 1. COMEbin: https://doi.org/10.1038/s41467-023-44290-z  ###"
echo "###############################################################"   
'''

# Split your samples into batches for parallel execution (b - number of samples per batch).
b = 5
batches = {f'b{i+1}-{i+b}': cleans[i:i + b] for i in range(0, len(cleans), b)}

#launch samples
for batch in batches:
    #if batch in [f'b1-{b}']:
    #    continue
        
    IDS = ' '.join(batches[batch]) 
    binac2(tasks=THREADS, time='10-00', mem=RAM, name=f'{batch}_COMEbin', logdir=logdir, 
           todo=todo.format(BATCH=batch, IDS=IDS, THREADS=THREADS, RAM=RAM))

Submitted batch job 125409
Submitted batch job 125410
Submitted batch job 125411
Submitted batch job 125412
Submitted batch job 125413
Submitted batch job 125414
Submitted batch job 125415
Submitted batch job 125416
Submitted batch job 125417
Submitted batch job 125418
Submitted batch job 125419
Submitted batch job 125420
Submitted batch job 125421
Submitted batch job 125422
Submitted batch job 125423
Submitted batch job 125424
Submitted batch job 125425
Submitted batch job 125426
Submitted batch job 125427
Submitted batch job 125428
Submitted batch job 125429
Submitted batch job 125430
Submitted batch job 125431
Submitted batch job 125432
Submitted batch job 125433
Submitted batch job 125434
Submitted batch job 125435
Submitted batch job 125436
Submitted batch job 125437
Submitted batch job 125438
Submitted batch job 125439


In [3]:
#check outputs 

total = 0
for clean in cleans:
    check = f'{PS}/COMEbin/{clean}/comebin_res/comebin_res_bins/0.fa'
    if not os.path.exists(check):
        print("Bins are not detected for sample:", clean)
        continue
    files = [c for c in os.listdir(f'{PS}/COMEbin/{clean}/comebin_res/comebin_res_bins') if c.endswith('.fa')]
    total += len(files)
print('Total bins before dereplication:', total)

Total bins before dereplication: 35811


## Evaluate bins with BUSCO

### Install BUSCO database

In [10]:
THREADS = 2
logdir = f'{LOGS}/BUSCO'

todo = '''
conda activate qiime2-moshpit-2025.4

busco --download bacteria_odb10

mkdir -p $DB/BUSCO
mv busco_downloads/lineages/bacteria_odb10 $DB/BUSCO
rm -r busco_downloads
'''

binac2(tasks=THREADS, time='01-00', mem=8, name=f'BUSCO_db', logdir=logdir, todo=todo)

Submitted batch job 128597


### Evaluate all bins assembled

In [22]:
THREADS = 12
logdir = f'{LOGS}/BUSCO'

todo = '''
conda activate qiime2-moshpit-2025.4

mkdir BUSCO
echo "Processing {BATCH}"
sras=({IDS})
for name in ${{sras[@]}}; do
    echo
    echo "############################################################"
    echo "Processing $name "

    bins=$PS/COMEbin/$name/comebin_res/comebin_res_bins
    busco -i $bins -m genome -c {THREADS} -o BUSCO/$name -l $DB/BUSCO/bacteria_odb10 --offline
    
done 
'''

# Split your samples into batches for parallel execution (b - number of samples per batch).
b = 10
batches = {f'b{i+1}-{i+b}': cleans[i:i + b] for i in range(0, len(cleans), b)}

#launch samples
for batch in batches:
    #if batch in [f'b1-{b}']:
    #    continue
        
    IDS = ' '.join(batches[batch]) 
    binac2(tasks=THREADS, time='2-00', mem=64, name=f'{batch}_BUSCO', logdir=logdir, 
           todo=todo.format(BATCH=batch, IDS=IDS, THREADS=THREADS))

Submitted batch job 128601
Submitted batch job 128602
Submitted batch job 128603
Submitted batch job 128604
Submitted batch job 128605
Submitted batch job 128606
Submitted batch job 128607
Submitted batch job 128608
Submitted batch job 128609
Submitted batch job 128610
Submitted batch job 128611
Submitted batch job 128612
Submitted batch job 128613
Submitted batch job 128614
Submitted batch job 128615


In [11]:
#!rm -r $WS/busco_downloads

### Summarize BUSCO runs
- completeness % = 100 x (1 - Missing / Total)
- contamination % = 100 x Duplicated / Complete

In [103]:
!mkdir -p {PS}/MAGs

busco = pd.DataFrame()
cols = ['Sample', 'MAG', 'Complete', 'Duplicated', 'Missing', 'Total']
for clean in cleans:
    with open(f"{WS}/BUSCO/{clean}/logs/busco.log", 'r') as batch:
        log = batch.read().split('Running BUSCO on file: ')[1:]
        for l in log:
            sample = l.split('COMEbin/')[-1].split('/')[0]
            mag = l.split('comebin_res_bins/')[-1].split('\n')[0]
            mis = l.split('(F)')[-1].split('Missing')[0].strip(' \n|')
            tot = l.split('(M)')[-1].split('Total')[0].strip(' \n|')
            dup = l.split('(S)')[-1].split('Complete')[0].strip(' \n|')
            com = l.split('Complete BUSCOs (C)')[0].split('|')[-1].strip(' \n|')
            busco.loc[len(busco), cols] = sample, mag, int(com), int(dup), int(mis), int(tot)

busco = busco.loc[(busco.Total > 0) & (busco.Complete > 0)]
busco['Completeness'] = 100 * (1 - busco.Missing/busco.Total)
busco['Contamination'] = 100 * busco.Duplicated/busco.Complete
busco['Completeness'] = busco.Completeness.astype(float).round(2)
busco['Contamination'] = busco.Contamination.astype(float).round(2)
busco.to_csv(f'{PS}/MAGs/busco_summary.tsv', sep='\t')
display(busco)

,Sample,MAG,Complete,Duplicated,Missing,Total,Completeness,Contamination
0,K10_21,34576.fa,58,0,48,124,61.29,0.00
3,K10_21,45450.fa,53,5,50,124,59.68,9.43
4,K10_21,53844.fa,10,0,95,124,23.39,0.00
5,K10_21,6820.fa,79,2,40,124,67.74,2.53
6,K10_21,48052.fa,1,0,122,124,1.61,0.00
...,...,...,...,...,...,...,...,...
35805,k16_26,1582.fa,119,0,3,124,97.58,0.00
35806,k16_26,51283.fa,1,0,122,124,1.61,0.00
35807,k16_26,6711.fa,1,0,122,124,1.61,0.00
35808,k16_26,3804.fa,3,0,121,124,2.42,0.00


## Pool MAGs (good quality bins)

### Select bins
#### You can modify completeness and contamination thresholds.
- cm - completeness threshold. Only bins with at least this value will be analysed.
- ct - contamination threshold- Only bins with contamination less that this value will be analysed.

In [22]:
cm = 50 #completeness
ct = 20 #contamination
busco = pd.read_csv(f'{PS}/MAGs/busco_summary.tsv', sep='\t', index_col=0)
good = busco.loc[(busco.Completeness >= cm) & (busco.Contamination <= ct)]
print(f'{len(good)} MAGs/bins with competeness >= {cm} and contamination < {ct}')
for clean in cleans:
    df = good.loc[good.Sample == clean]
    with open(f'{PS}/COMEbin/{clean}/good', 'w') as file:
        file.write('\n'.join(df.MAG.tolist()))
good.to_csv(f'{PS}/MAGs/busco_summary_good.tsv', sep='\t')

12388 MAGs/bins with competeness >= 50 and contamination <= 20


### Copy selected bins

In [15]:
THREADS = 2
logdir = f'{LOGS}/Pool_bins'

todo = '''
pooled=$PS/Bins_pooled

echo "Processing {BATCH}"
sras=({IDS})
for name in ${{sras[@]}}; do
    echo
    echo "############################################################"
    echo "Processing $name "

    mkdir -p $pooled/$name
    bins=$PS/COMEbin/$name/comebin_res/comebin_res_bins
    rsync -a $bins --files-from=$bins/../../good $pooled/$name
    cd $pooled/$name
    for fa in ./*.fa; do
        f=$(basename $fa)
        mv $fa ${{name}}_${{f}}
    done
    
done 
'''

# Split your samples into batches for parallel execution (b - number of samples per batch).
b = 10
batches = {f'b{i+1}-{i+b}': cleans[i:i + b] for i in range(0, len(cleans), b)}

#launch samples
for batch in batches:
    #if batch in f'b1-{b}']:
    #    continue
     
    IDS = ' '.join(batches[batch])
    binac2(tasks=THREADS, time='1-00', mem=8, name=f'{batch}_cp_bins', logdir=logdir, 
           todo=todo.format(BATCH=batch, IDS=IDS))

Submitted batch job 130961
Submitted batch job 130962
Submitted batch job 130963
Submitted batch job 130964
Submitted batch job 130965
Submitted batch job 130966
Submitted batch job 130967
Submitted batch job 130968
Submitted batch job 130969
Submitted batch job 130970
Submitted batch job 130971
Submitted batch job 130972
Submitted batch job 130973
Submitted batch job 130974
Submitted batch job 130975
Submitted batch job 130976


In [21]:
# Should be equal to the number of selected bins in "Select bins" cell output

files = []
for folder in os.listdir(f'{PS}/Bins_pooled/'):
    if not folder.startswith('.'):
        [files.append(c) for c in os.listdir(f'{PS}/Bins_pooled/{folder}/') if c.endswith('.fa')]
    
print('Total retained bins/MAGs, pooled:', len(files))

Total retained bins/MAGs, pooled: 12388


In [14]:
#!rm -r $PS/Bins_pooled

## Import bins to Qiime2

### Manifest and renaming bins according UUID4 convention

In [4]:
good = pd.read_csv(f'{PS}/MAGs/busco_summary_good.tsv', sep='\t', index_col=0)
mf = pd.DataFrame()
for clean in cleans:
    path = f'{PS}/Bins_pooled/{clean}/'
    for mag in os.listdir(path):
        if mag.endswith('.fa'):
            random = uuid4()
            sample, file = mag.rsplit('_', 1)
            os.rename(os.path.join(path, mag), os.path.join(path, f'{random}.fasta'))
            good.loc[((good['Sample'] == sample) & (good['MAG'] == file)), 'MAG_UUID4'] = random
            mf.loc[len(mf), ['sample-id', 'mag-id', 'filename']] = clean, random, f'{clean}/{random}.fasta'

mf.to_csv(f'{PS}/Bins_pooled/MANIFEST', index=False)
good.to_csv(f'{PS}/MAGs/busco_summary_good.tsv', sep='\t')
display(mf, good)

,sample-id,mag-id,filename
0,K10_21,67ef6e7a-82dd-452b-8ad7-b6dd16230569,K10_21/67ef6e7a-82dd-452b-8ad7-b6dd16230569.fasta
1,K10_21,93b36881-56d0-4e2b-80dc-37b61f8b4f06,K10_21/93b36881-56d0-4e2b-80dc-37b61f8b4f06.fasta
2,K10_21,16570994-a950-49e0-a213-851a6508f8cd,K10_21/16570994-a950-49e0-a213-851a6508f8cd.fasta
3,K10_21,3c597cd4-1d0d-4004-bee0-5de4749d9abc,K10_21/3c597cd4-1d0d-4004-bee0-5de4749d9abc.fasta
4,K10_21,3cc3a4b9-b996-449f-9ef4-f82190ec969c,K10_21/3cc3a4b9-b996-449f-9ef4-f82190ec969c.fasta
...,...,...,...
12383,k16_26,2d3f236e-a3c1-401f-ab5f-867c3c6294f3,k16_26/2d3f236e-a3c1-401f-ab5f-867c3c6294f3.fasta
12384,k16_26,72974340-62a7-40bf-83f6-a103b01b45f1,k16_26/72974340-62a7-40bf-83f6-a103b01b45f1.fasta
12385,k16_26,b07780c6-d70a-4204-a569-18e11121e03a,k16_26/b07780c6-d70a-4204-a569-18e11121e03a.fasta
12386,k16_26,0efbf568-a888-4d4e-bef3-c0e45182495f,k16_26/0efbf568-a888-4d4e-bef3-c0e45182495f.fasta


,Sample,MAG,Complete,Duplicated,Missing,Total,Completeness,Contamination,MAG_UUID4
0,K10_21,34576.fa,58,0,48,124,61.29,0.00,014d8763-2f77-41f3-8152-a2dc66feed48
3,K10_21,45450.fa,53,5,50,124,59.68,9.43,aa91a51d-77e3-4a3a-8dad-d2b2a15171d6
5,K10_21,6820.fa,79,2,40,124,67.74,2.53,7ed0fe7f-52ed-4771-b3ce-169e10c9b45f
8,K10_21,19559.fa,75,1,36,124,70.97,1.33,d9838e66-b6cf-44c1-a0a0-6bf9c3e6cd7d
12,K10_21,2224.fa,83,1,34,124,72.58,1.20,a9a506be-18eb-4024-a1f7-3a1a7a1727e8
...,...,...,...,...,...,...,...,...,...
35786,k16_26,2149.fa,112,3,10,124,91.94,2.68,5b16aff9-a851-48cd-b047-27482aa4f998
35787,k16_26,17160.fa,48,1,62,124,50.00,2.08,50a74398-7a90-426a-bc95-5ae0850465a0
35790,k16_26,16892.fa,111,1,8,124,93.55,0.90,2045d469-1406-444c-885c-ce1e604e2886
35805,k16_26,1582.fa,119,0,3,124,97.58,0.00,b2225b73-2e56-4f59-941b-4303f89cbfbd


### Import

In [5]:
THREADS = 2
logdir = f'{LOGS}/Q2'

todo = '''
conda activate qiime2-moshpit-2025.4

rm -r $PS/Bins_pooled/.ipynb_* $PS/Bins_pooled/*/.ipynb_*

qiime tools cache-import \
    --type 'SampleData[MAGs]' \
    --input-path $PS/Bins_pooled \
    --input-format 'MultiMAGSequencesDirFmt' \
    --cache ./cache \
    --key 'pooled_bins'
'''

binac2(tasks=THREADS, time='02-00', mem=16, name=f'MAGs_Q2_imp', logdir=logdir, todo=todo)

Submitted batch job 130990


## Dereplicate MAGs

In [6]:
THREADS = 12
logdir = f'{LOGS}/Q2'

todo = '''
conda activate qiime2-moshpit-2025.4

#compute minhashes
qiime sourmash compute \
    --i-sequence-file ./cache:pooled_bins \
    --p-ksizes 35 \
    --p-scaled 10 \
    --o-min-hash-signature ./cache:mags_minhash \
    --verbose

#compare minhashes
qiime sourmash compare \
    --i-min-hash-signature ./cache:mags_minhash \
    --p-ksize 35 \
    --o-compare-output ./cache:mags_dist_matrix \
    --verbose

#dereplicate MAGs
qiime annotate dereplicate-mags \
    --i-mags ./cache:pooled_bins \
    --i-distance-matrix ./cache:mags_dist_matrix \
    --p-threshold 0.99 \
    --o-dereplicated-mags $PS/MAGs/derep_mags.qza \
    --o-table $PS/MAGs/derep_mags_table.qza \
    --verbose

qiime feature-table summarize \
    --i-table $PS/MAGs/derep_mags_table.qza \
    --o-visualization $PS/MAGs/derep_mags_table.qzv
'''

binac2(tasks=THREADS, time='10-00', mem=120, name=f'MAGs_dereplicate', logdir=logdir, todo=todo)

Submitted batch job 130991


In [28]:
!rm -r $PS/MAGs_Test

## MAGs taxonomy

In [2]:
THREADS = 24
logdir = f'{LOGS}/Q2'

todo = '''
conda activate qiime2-moshpit-2025.4

#taxonomy with kraken2
qiime annotate classify-kraken2 \
    --i-seqs $PS/MAGs/derep_mags.qza \
    --i-db $DB/{K2DB}/kraken2.qza \
    --p-threads {THREADS} \
    --p-confidence 0.1 \
    --p-report-minimizer-data \
    --o-reports ./cache:kraken_reports_mags_derep \
    --o-outputs ./cache:kraken_hits_mags_derep \
    --verbose
    
#convert to q2 taxonomy
qiime annotate kraken2-to-mag-features \
    --i-reports ./cache:kraken_reports_mags_derep  \
    --i-outputs ./cache:kraken_hits_mags_derep  \
    --o-taxonomy $PS/Taxonomy/taxonomy_mags.qza \
    --verbose
'''

binac2(tasks=THREADS, time='03-00', mem=120, name=f'MAGs_taxonomy', logdir=logdir, 
       todo=todo.format(THREADS=THREADS))

Submitted batch job 317892


### ATTENTION!
If you got an error at MAG taxonomy step "['r1__cellular organisms'] not in index", then run this cell.

In [29]:
with open(f"{WS}/cache/keys/kraken_reports_mags_derep") as txt:
    key = txt.read().split(' ', 1)[-1].split('\n')[0]
    loc = f"{WS}/cache/data/{key}/data"
    
clean = !rm -r {WS}/cache/*/*/*/.ipynb* {WS}/cache/*/*/.ipynb* {WS}/cache/*/.ipynb* {WS}/cache/.ipynb*

for rep in os.listdir(f"{WS}/cache/data/{key}/data"):
    with open(f"{loc}/{rep}") as report:
        report = report.read()
        if "cellular organisms" in report and len(report.split('\n')) == 4:
            !rm {loc}/{rep}

THREADS = 8
logdir = f'{LOGS}/Q2'

todo = '''
conda activate qiime2-moshpit-2025.4
    
#convert to q2 taxonomy
qiime annotate kraken2-to-mag-features \
    --i-reports ./cache:kraken_reports_mags_derep  \
    --i-outputs ./cache:kraken_hits_mags_derep  \
    --o-taxonomy $PS/Taxonomy/taxonomy_mags.qza \
    --verbose
'''

binac2(tasks=THREADS, time='01-00', mem=32, name=f'MAGs_taxonomy_fix', logdir=logdir, 
       todo=todo.format(THREADS=THREADS))

Submitted batch job 443004


## MAG abundances

### Export dereplicated MAGs

In [31]:
THREADS = 2
logdir = f'{LOGS}/Q2'

todo = '''
conda activate qiime2-moshpit-2025.4

mkdir -p $PS/MAGs/Exported
qiime tools export \
    --input-path $PS/MAGs/derep_mags.qza \
    --output-path $PS/MAGs/Exported
'''

binac2(tasks=THREADS, time='01-00', mem=16, name=f'Derep_MAGs_export', logdir=logdir, todo=todo)

Submitted batch job 443006


In [30]:
#!rm -r $PS/MAGs/Exported {PS}/MAGs/Filtered_out

### Filter based on taxonomy (optional)
There are some MAGs that unassigned to bacteria or archaea and can be the residues of the host or feed DNA. They lead to the high presence of functions that are not of microbial origin. Interpretaton of such functions can be misleading.

#### Export taxonomy

In [41]:
THREADS = 2
logdir = f'{LOGS}/Q2'

todo = '''
conda activate qiime2-moshpit-2025.4

qiime tools export \
    --input-path $PS/Taxonomy/taxonomy_mags.qza \
    --output-path $PS/MAGs
'''

binac2(tasks=THREADS, time='01-00', mem=8, name=f'Tax_MAGs_export', logdir=logdir, todo=todo)

Submitted batch job 443007


#### Copy MAGs that are not from bacteria/archaea to another directory

In [42]:
taxa = pd.read_csv(f'{PS}/MAGs/taxonomy.tsv', sep='\t', index_col=0)
trash = taxa.loc[(~taxa.Taxon.str.contains('d__Bac')) & (~taxa.Taxon.str.contains('d__Arc'))]

!mkdir {PS}/MAGs/Filtered_out

for ind in trash.index:
    
    !mv {PS}/MAGs/Exported/{ind}.fa* {PS}/MAGs/Filtered_out/

### Import back as whole table

In [19]:
THREADS = 2
logdir = f'{LOGS}/Q2'

todo = '''
conda activate qiime2-moshpit-2025.4

qiime tools import \
    --output-path $PS/MAGs/derep_mags.qza \
    --type "FeatureData[MAG]" \
    --input-path $PS/MAGs/Exported
'''

binac2(tasks=THREADS, time='01-00', mem=16, name=f'Derep_MAGs_Q2_reimport', 
       logdir=logdir, todo=todo)

Submitted batch job 600489


### Import back as batches

In [43]:
THREADS = 2
logdir = f'{LOGS}/Q2'

todo = '''
conda activate qiime2-moshpit-2025.4

mkdir -p $WS/{BATCH}

IDS=({IDS})
for fa in ${{IDS[@]}}; do
    cp $PS/MAGs/Exported/$fa $WS/{BATCH}/$fa
done

qiime tools cache-import \
    --cache ./cache \
    --key {BATCH}_derep_mags \
    --type "FeatureData[MAG]" \
    --input-path $WS/{BATCH}

#rm -r $WS/{BATCH}
'''

# Split your MAGs into batches for parallel execution (b - number of MAGs per batch).
mags = [c for c in os.listdir(f'{PS}/MAGs/Exported') if c.endswith('.fasta')]
b = 100
batches = {f'b{i+1}-{i+b}': mags[i:i + b] for i in range(0, len(mags), b)}

#launch samples
for batch in batches:
    #if batch in [f'b1-{b}']:
    #    continue

    IDS = ' '.join(batches[batch])
    binac2(tasks=THREADS, time='01-00', mem=16, name=f'{batch}_Derep_MAGs_Q2_imp', logdir=logdir, 
           todo=todo.format(IDS=IDS, BATCH=batch))

Submitted batch job 443008
Submitted batch job 443009
Submitted batch job 443010
Submitted batch job 443011
Submitted batch job 443012
Submitted batch job 443013
Submitted batch job 443014
Submitted batch job 443015
Submitted batch job 443016
Submitted batch job 443017
Submitted batch job 443018
Submitted batch job 443019
Submitted batch job 443020
Submitted batch job 443021
Submitted batch job 443022
Submitted batch job 443023
Submitted batch job 443024
Submitted batch job 443025
Submitted batch job 443026
Submitted batch job 443027
Submitted batch job 443028


### MAGs length and index

In [29]:
THREADS = 12
logdir = f'{LOGS}/Q2'

todo = '''
conda activate qiime2-moshpit-2025.4

#MAGs lengths
qiime annotate get-feature-lengths \
    --i-features $PS/MAGs/derep_mags.qza \
    --o-lengths ./cache:mags_derep_length \
    --verbose                  

#index dereplicated MAGs
qiime assembly index-derep-mags \
    --i-mags $PS/MAGs/derep_mags.qza \
    --p-threads {THREADS} \
    --p-seed 100 \
    --o-index ./cache:mags_derep_index \
    --verbose
'''

binac2(tasks=THREADS, time='01-00', mem=64, name=f'MAGs_length', logdir=logdir, 
       todo=todo.format(THREADS=THREADS))

Submitted batch job 600792


### Get abundances (in batches)

In [42]:
THREADS = 24
logdir = f'{LOGS}/Q2'

todo = '''
conda activate qiime2-moshpit-2025.4
mkdir -p TEMP/{BATCH}
export TMPDIR=$WS/TEMP/{BATCH}

#Map MAGs to reads
qiime assembly map-reads \
    --i-index ./cache:mags_derep_index \
    --i-reads ./cache:{BATCH} \
    --p-threads {THREADS} \
    --p-seed 100 \
    --o-alignment-maps ./cache:{BATCH}_reads_to_derep_mags \
    --verbose

#Get abundances
qiime annotate estimate-abundance \
    --i-feature-lengths ./cache:mags_derep_length \
    --i-alignment-maps ./cache:{BATCH}_reads_to_derep_mags \
    --p-threads {THREADS} \
    --p-metric tpm \
    --p-min-mapq 42 \
    --o-abundances $PS/MAGs/{BATCH}_mags_table_tpm.qza \
    --verbose

rm -r $WS/TEMP/{BATCH}
'''

# Split your samples into batches for parallel execution (b - number of samples per batch).
b = 10
batches = {f'b{i+1}-{i+b}': cleans[i:i + b] for i in range(0, len(cleans), b)}

#launch samples
for batch in batches:
    #if batch in [f'b1-{b}']:
    #    continue
    
    if os.path.exists(f'{PS}/MAGs/{batch}_mags_table_tpm.qza'):
        continue
    
    binac2(tasks=THREADS, time='5-00', mem=256, name=f'{batch}_MAG_tpm', logdir=logdir, 
           todo=todo.format(BATCH=batch, THREADS=THREADS))

Submitted batch job 600811
Submitted batch job 600812
Submitted batch job 600813
Submitted batch job 600814
Submitted batch job 600815
Submitted batch job 600816
Submitted batch job 600817
Submitted batch job 600818
Submitted batch job 600819
Submitted batch job 600820
Submitted batch job 600821
Submitted batch job 600822
Submitted batch job 600823
Submitted batch job 600824
Submitted batch job 600825
Submitted batch job 600826


### Merge MAG tables

In [3]:
THREADS = 2
logdir = f'{LOGS}/Q2'

todo = '''
conda activate qiime2-moshpit-2025.4

qiime feature-table merge \
    --i-tables $PS/MAGs/b*-*_mags_table_tpm.qza \
    --o-merged-table $PS/MAGs/mags_table_tpm.qza

qiime feature-table summarize \
    --i-table $PS/MAGs/mags_table_tpm.qza \
    --o-visualization $PS/MAGs/mags_table_tpm.qzv

#rm $PS/MAGs/b*-*_mags_table_tpm.qza
'''

#launch samples             
binac2(tasks=THREADS, time='01-00', mem=16, name=f'Merging_tables_mags', logdir=logdir, todo=todo)

Submitted batch job 744742


# Functional annotations

## Fetch databases

### Diamond

In [19]:
THREADS = 2
logdir = f'{LOGS}/Q2'

todo = '''
conda activate qiime2-moshpit-2025.4

mkdir -p $DB/Diamond

qiime annotate fetch-diamond-db \
    --o-db $DB/Diamond/diamond_db.qza \
    --verbose
'''

binac2(tasks=THREADS, time='02-00', mem=16, name=f'Diamond_DB', logdir=logdir, todo=todo)

Submitted batch job 128797


### EggNOG

In [22]:
THREADS = 2
logdir = f'{LOGS}/Q2'

todo = '''
conda activate qiime2-moshpit-2025.4

mkdir -p $DB/EggNOG

qiime annotate fetch-eggnog-db \
    --o-db $DB/EggNOG/eggnog_db.qza \
    --verbose
'''

binac2(tasks=THREADS, time='02-00', mem=16, name=f'EggNOG_DB', logdir=logdir, todo=todo)

Submitted batch job 128799


## Get functions
Since dereplicated MAGs will require a long time for functional annotations, lets export them and reimport into Qiime2 in batches for parallel execution

### EggNOG annotations

In [4]:
THREADS = 12
logdir = f'{LOGS}/Q2'

todo = '''
conda activate qiime2-moshpit-2025.4

mkdir -p $PS/Functions

#diamond
qiime annotate search-orthologs-diamond \
    --i-seqs ./cache:{BATCH}_derep_mags \
    --i-db $DB/Diamond/diamond_db.qza \
    --p-num-cpus {THREADS} \
    --p-db-in-memory \
    --o-eggnog-hits $PS/Functions/{BATCH}_eggnog_hits.qza \
    --o-table $PS/Functions/{BATCH}_eggnog_table.qza  \
    --o-loci $PS/Functions/{BATCH}_eggnog_loci.qza \
    --verbose

#eggnog
qiime annotate map-eggnog \
    --i-eggnog-hits $PS/Functions/{BATCH}_eggnog_hits.qza \
    --i-db $DB/EggNOG/eggnog_db.qza \
    --p-num-cpus {THREADS} \
    --p-db-in-memory \
    --o-ortholog-annotations $PS/Functions/{BATCH}_eggnog_annotations.qza \
    --verbose
'''

# Split your MAGs into batches for parallel execution (b - number of MAGs per batch).
mags = [c for c in os.listdir(f'{PS}/MAGs/Exported') if c.endswith('.fasta')]
b = 100
batches = {f'b{i+1}-{i+b}': mags[i:i + b] for i in range(0, len(mags), b)}

#launch samples
for batch in batches:
    #if batch in [f'b1-{b}']:
    #    continue

    binac2(tasks=THREADS, time='10-00', mem=64, name=f'{batch}_Func_Eggnog', logdir=logdir, 
           todo=todo.format(THREADS=THREADS, BATCH=batch))

Submitted batch job 501358
Submitted batch job 501359
Submitted batch job 501360
Submitted batch job 501361
Submitted batch job 501362
Submitted batch job 501363
Submitted batch job 501364
Submitted batch job 501365
Submitted batch job 501366
Submitted batch job 501367
Submitted batch job 501368
Submitted batch job 501369
Submitted batch job 501370
Submitted batch job 501371
Submitted batch job 501372
Submitted batch job 501373
Submitted batch job 501374
Submitted batch job 501375
Submitted batch job 501376
Submitted batch job 501377
Submitted batch job 501378


In [3]:
mags = [c for c in os.listdir(f'{PS}/MAGs/Exported') if c.endswith('.fasta')]
b = 100
batches = {f'b{i+1}-{i+b}': mags[i:i + b] for i in range(0, len(mags), b)}
len(batches)

21

### Merge EggNOG tables

In [8]:
THREADS = 2
logdir = f'{LOGS}/Q2'

todo = '''
conda activate qiime2-moshpit-2025.4

qiime feature-table merge \
    --i-tables $PS/Functions/b*-*_eggnog_table.qza \
    --p-overlap-method average \
    --o-merged-table $PS/Functions/eggnog_table.qza

qiime feature-table summarize \
    --i-table $PS/Functions/eggnog_table.qza \
    --o-visualization $PS/Functions/eggnog_table.qzv

#rm $PS/Functions/b*-*_eggnog_table.qza
'''

#launch samples             
binac2(tasks=THREADS, time='01-00', mem=16, name=f'Merging_tables_eggnog', logdir=logdir, todo=todo)

Submitted batch job 600255


In [7]:
!qiime feature-table merge

/bin/bash: line 1: qiime: command not found


### Export EggNOG batches

In [5]:
THREADS = 2
logdir = f'{LOGS}/Q2'

todo = '''
conda activate qiime2-moshpit-2025.4

mkdir -p $PS/Functions/Annotations
qiime tools export \
    --input-path $PS/Functions/{BATCH}_eggnog_annotations.qza \
    --output-path $PS/Functions/Annotations

mkdir -p $PS/Functions/Hits
qiime tools export \
    --input-path $PS/Functions/{BATCH}_eggnog_hits.qza \
    --output-path $PS/Functions/Hits

mkdir -p $PS/Functions/Loci
qiime tools export \
    --input-path $PS/Functions/{BATCH}_eggnog_loci.qza \
    --output-path $PS/Functions/Loci    
'''

# Split your MAGs into batches for parallel execution (b - number of MAGs per batch).
mags = [c for c in os.listdir(f'{PS}/MAGs/Exported') if c.endswith('.fasta')]
b = 100
batches = {f'b{i+1}-{i+b}': mags[i:i + b] for i in range(0, len(mags), b)}

#launch samples
for batch in batches:
    #if batch in [f'b1-{b}']:
    #    continue

    IDS = ' '.join(batches[batch])
    binac2(tasks=THREADS, time='01-00', mem=16, name=f'{batch}_exp_egg', logdir=logdir, 
           todo=todo.format(IDS=IDS, BATCH=batch))

Submitted batch job 600234
Submitted batch job 600235
Submitted batch job 600236
Submitted batch job 600237
Submitted batch job 600238
Submitted batch job 600239
Submitted batch job 600240
Submitted batch job 600241
Submitted batch job 600242
Submitted batch job 600243
Submitted batch job 600244
Submitted batch job 600245
Submitted batch job 600246
Submitted batch job 600247
Submitted batch job 600248
Submitted batch job 600249
Submitted batch job 600250
Submitted batch job 600251
Submitted batch job 600252
Submitted batch job 600253
Submitted batch job 600254


### Import back EggNOG batches as whole dataset

In [12]:
THREADS = 2
logdir = f'{LOGS}/Q2'

todo = '''
conda activate qiime2-moshpit-2025.4

qiime tools import \
    --type "GenomeData[NOG]" \
    --input-path $PS/Functions/Annotations \
    --output-path $PS/Functions/eggnog_annotations.qza

qiime tools import \
    --type "SampleData[Orthologs]" \
    --input-path $PS/Functions/Hits \
    --output-path $PS/Functions/eggnog_hits.qza

qiime tools import \
    --type "GenomeData[Loci]" \
    --input-path $PS/Functions/Loci \
    --output-path $PS/Functions/eggnog_loci.qza
'''

binac2(tasks=THREADS, time='01-00', mem=16, name=f'imp_egg', logdir=logdir, todo=todo)

Submitted batch job 600256


In [20]:
#make sure that all commands above work and clean the room!

!rm -r {PS}/Functions/Annotations {PS}/Functions/Hits {PS}/Functions/b*-*_*.qza

In [3]:
#qiime2 2025.4

THREADS = 12
logdir = f'{LOGS}/Q2'

todo = '''
conda activate qiime2-moshpit-2024.10

mkdir -p $PS/Functions

#diamond
qiime annotate search-orthologs-diamond \
    --i-sequences $PS/MAGs/derep_mags.qza \
    --i-diamond-db $DB/Diamond/diamond_db.qza \
    --p-num-cpus {THREADS} \
    --p-db-in-memory \
    --o-eggnog-hits ./cache:eggnog_hits \
    --o-table $PS/Functions/eggnog_table.qza  \
    --verbose

#eggnog
qiime annotate map-eggnog \
    --i-eggnog-hits ./cache:eggnog_hits \
    --i-eggnog-db ./cache:eggnog_db \
    --p-num-cpus {THREADS} \
    --p-db-in-memory \
    --o-ortholog-annotations $PS/Functions/eggnog_annotations.qza \
    --verbose
'''

binac2(tasks=THREADS, time='05-00', mem=120, name=f'Func_Eggnog', logdir=logdir, 
       todo=todo.format(THREADS=THREADS))

Submitted batch job 131242


### Convert to other categories

In [11]:
keep = pd.DataFrame()
keep.index = [f.rsplit('.', 1)[0] for f in os.listdir(f'{PS}/MAGs/Exported') if f.endswith('.fasta')]
keep.index.name = 'SampleID'
keep.to_csv(f'{PS}/MAGs/keep.tsv', sep='\t')
len(keep)

2046

In [12]:
THREADS = 2
logdir = f'{LOGS}/Q2'

todo = '''
conda activate qiime2-moshpit-2025.4

#extract categories
!qiime feature-table filter-samples \
    --i-table $PS/Functions/eggnog_annotations.qza \
    --m-metadata-file $PS/MAGs/keep.tsv \
    --o-filtered-table {$PS/Functions/eggnog_annotations.qza
'''


binac2(tasks=THREADS, time='01-00', mem=32, name=f'Filter_eggs', logdir=logdir, todo=todo)

Submitted batch job 771740


In [21]:
THREADS = 2
logdir = f'{LOGS}/Q2'

todo = '''
conda activate qiime2-moshpit-2025.4

#extract categories
#qiime annotate extract-annotations \
#    --i-ortholog-annotations $PS/Functions/eggnog_annotations.qza \
#    --p-annotation {CAT} \
#    --p-max-evalue 0.0001 \
#    --o-annotation-frequency $PS/Functions/{CAT}_annotations.qza \
#    --verbose

#filter tables
qiime feature-table filter-samples \
    --i-table $PS/Functions/{CAT}_annotations.qza \
    --m-metadata-file $PS/MAGs/keep.tsv \
    --o-filtered-table $PS/Functions/{CAT}_annotations.qza

#multiply categories
qiime annotate multiply-tables \
    --i-table1 $PS/MAGs/mags_table_tpm.qza \
    --i-table2 $PS/Functions/{CAT}_annotations.qza \
    --o-result-table $PS/Functions/{CAT}_table.qza \
    --verbose

qiime feature-table summarize \
    --i-table $PS/Functions/{CAT}_table.qza \
    --o-visualization $PS/Functions/{CAT}_table.qzv
'''

#select categories to convert into here!
cats = ['cog', 'caz', 'kegg_ko', 'kegg_pathway', 'kegg_reaction', 'kegg_module', 'brite']

for cat in cats:
    if cat in ['kegg_reaction',]:
        continue
    binac2(tasks=THREADS, time='01-00', mem=32, name=f'Egg_to_{cat}', logdir=logdir, 
           todo=todo.format(CAT=cat))

sbatch: error: Batch job submission failed: Socket timed out on send/recv operation
Submitted batch job 772321
Submitted batch job 772323
Submitted batch job 772335
Submitted batch job 772352
Submitted batch job 772367


# Status

In [27]:
!squeue --Format $FORMAT -u $USER

JOBID     NAME                MIN_MEMORY TASKS STATE    TIME        TIME_LIMIT 
772335    Egg_to_kegg_pathway 32G        2     RUNNING  1:31        1-00:00:00 
772352    Egg_to_kegg_module  32G        2     RUNNING  1:31        1-00:00:00 
772367    Egg_to_brite        32G        2     RUNNING  1:31        1-00:00:00 
772323    Egg_to_kegg_ko      32G        2     RUNNING  1:35        1-00:00:00 
772321    Egg_to_caz          32G        2     RUNNING  1:37        1-00:00:00 
772320    Egg_to_cog          32G        2     RUNNING  1:38        1-00:00:00 


In [24]:
!scancel 744914

In [ ]:
!scontrol show job 201765